# Kamodo Model/Data Comparison Notebook

## TIEGCM Model/Data comparison
#### Start with extracting satellite positions from model output

In [ ]:
# Import some libraries
from kamodo import Kamodo
import kamodo_ccmc.flythrough.model_wrapper as MW
from kamodo_ccmc.flythrough.model_wrapper import Choose_Model
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
from kamodo_ccmc.flythrough.plots import SatPlot4D
from kamodo_ccmc.readers.hapi import HAPI
from kamodo_ccmc.flythrough.wrapper_output import Functionalize_TimeSeries

In [ ]:
%%bash
touch ModelFlythrough_TIEGCM
rm -f ModelFlythrough_TIEGCM*
# Flythrough complains if overwriting files, so make sure they are gone first. BE CAREFUL HERE

In [ ]:
# Set some values for the next few steps
# NOTE: The TIEGCM data from this run is stored locally (2GB)
model='TIEGCM'
file_dir = '/Users/ddezeeuw/KAMODO_DEMO/TIE-GCM/Uriel_Ramirez_012517_IT_1/'

In [ ]:
# Show the time periods covered by the model output we are using
MW.File_Times(model, file_dir)

In [ ]:
# Show the variables included in the model output
MW.File_Variables(model, file_dir)

In [ ]:
# Set input values for RealFlight function call
dataset = 'cnofs'
start_utcts, end_utcts = 1426638000, 1426665600
variable_list = ['T_i'] #list of desired variable names from above list, must be in list form
coord_type = 'GEO'  #GEO cartesian coordinates as the sample coordinate system for trajectory.
output_name = 'ModelFlythrough_TIEGCM.csv' #filename for DATA output with extension
plot_coord = 'GSE'  #coordinate system chosen for output plots

In [ ]:
#run RealFlight with cnofs satellite trajectory (CINDI)
results = SF.RealFlight(dataset, start_utcts, end_utcts, model, file_dir, variable_list, coord_type, 
                       output_name=output_name, plot_coord=plot_coord)
#open plots in separate internet browser window for interactivity. Nothing will open here.

In [ ]:
# The results from the satellite extraction can be plugged into Kamodo easily
kamodo_object = SF.WO.Functionalize_SFResults(model,results)
kamodo_object

In [ ]:
# Done with results, delete to save memory
del results

In [ ]:
# Using Kamodo to generate a plot of the chosen variable
kamodo_object.plot('T_i')

#### Get CINDI data via HAPI server from CDAWeb

In [ ]:
# Set details of data and grab it
server = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset = 'CNOFS_CINDI_IVM_500MS'
parameters = 'ionTemperature'
start      = '2015-03-18T00:20:00'
stop       = '2015-03-18T08:00:00'

hapiCDA = HAPI(server, dataset, parameters, start, stop)

In [ ]:
# Plot The values
hapiCDA.plot('ionTemperature')

In [ ]:
# You can copy data from one Kamodo object into another to plot together
kamodo_object['T_iCINDI']=hapiCDA['ionTemperature']

In [ ]:
# Interpolate model data T_i onto same time series as observed data
interpT_i = kamodo_object.T_i(hapiCDA.dtarray)

# Compute difference of two identical length arrays
deltaT_i = hapiCDA.variables['ionTemperature']['data'] - interpT_i

# Add the new time series back into the Kamodo object for further analysis/plotting
kamodo_object = Functionalize_TimeSeries(hapiCDA.tsarray, 'DIFFERENCE', 'K', deltaT_i, kamodo_object=kamodo_object)

In [ ]:
# Now we can plot model and data on the same figure with the difference
kamodo_object.plot('T_i','T_iCINDI','DIFFERENCE')